In [1]:
# imports
from astropy.io import fits
# import matplotlib.pyplot as plt
import numpy as np
import csv 
import match

In [2]:
# path variables 
path_cat = 'D:\IfA REU\Catalogs'
path_fig = 'D:\IfA REU\IfA-REU-Project\Figures'
path_csv = 'D:\IfA REU\IfA-REU-Project\Lists'

# Get AGN IDs

In [3]:
# read from a given csv file and return a list of its contents 
def ReadFile(filename) : 
    # initialize list
    file = []

    # open csv file
    with open(path_csv+filename, newline='') as csv_file : 
        reader = csv.reader(csv_file, delimiter=',', quotechar='"')
        # output file into list for easy access
        for row in reader :
            file.append(row)

    # print column names 
    print(filename, ' columns: ', file[0])

    # return an array of type int with column names clipped off
    return( np.array(file[1:], dtype=int) )


In [4]:
# read AGN ID and index from csv files
agn_irSel = ReadFile('\Match_COSMOS2020_IRAGN_to_COSMOS2020.csv')
agn_xrSel = ReadFile('\Match_Chandra2016_XrayAGN_to_COSMOS2020.csv')

# transpose arrays to get columns
agn_irSel_id, agn_irSel_i = agn_irSel.T
agn_xrSel_id, agn_xrSel_i = agn_xrSel.T

\Match_COSMOS2020_IRAGN_to_COSMOS2020.csv  columns:  ['ID', 'i']
\Match_Chandra2016_XrayAGN_to_COSMOS2020.csv  columns:  ['ID', 'i']


In [15]:
# print info
print('Number of IR AGN:\t',    len(agn_irSel_id))
print('Number of Xray AGN:\t',  len(agn_xrSel_id))

Number of IR AGN:	 2287
Number of Xray AGN:	 2176


In [5]:
# # test: left and right must match
# print(agn_irSel[0][0], '==', agn_irSel_id[0])
# print(agn_irSel[0][1], '==', agn_irSel_i[0])
# print(agn_xrSel[0][0], '==', agn_xrSel_id[0])
# print(agn_xrSel[0][1], '==', agn_xrSel_i[0])


In [16]:
# match agn ids
key_ir, key_xr = match.match(agn_irSel_id, agn_xrSel_id)

# initialize
mask_ir = np.ones(agn_irSel_id.size, dtype=bool)
mask_xr = np.ones(agn_xrSel_id.size, dtype=bool)

# set matched index to false
mask_ir[key_ir] = False
mask_xr[key_xr] = False

# remove xray selected agn from ir selected list
agn_irSel_id_ONLY = agn_irSel_id[mask_ir]
agn_irSel_i_ONLY = agn_irSel_i[mask_ir]

# remove ir selected agn from xray selected list 
agn_xrSel_id_ONLY = agn_xrSel_id[mask_xr]
agn_xrSel_i_ONLY = agn_xrSel_i[mask_xr]

# create list of ir/xray match
agn_id_BOTH = agn_irSel_id[key_ir]
agn_i_BOTH = agn_irSel_i[key_ir]


In [24]:
# print info 
print('Number of IR only:\t',   len(agn_irSel_i_ONLY))
print('Number of Xray only:\t', len(agn_xrSel_i_ONLY))
print('Number of matches:\t',   len(agn_i_BOTH))

Number of IR only:	 1640
Number of Xray only:	 1529
Number of matches:	 647


# Get Data From Catalogs

In [7]:
# open COSMOS 2020 catalog file and get data 
inf20 = fits.open(path_cat+'\COSMOS2020_CLASSIC_R1_v2.0_master.fits')
data20 = inf20[1].data
cols20 = inf20[1].columns
inf20.close()

In [8]:
# get IRAC data (unit = 'uJy')
ch1_data  = np.array(data20['SPLASH_CH1_FLUX']) # 3.5686 um
print('Ch1 read')
ch2_data  = np.array(data20['SPLASH_CH2_FLUX']) # 4.5067 um
print('Ch2 read')
ch3_data  = np.array(data20['SPLASH_CH3_FLUX']) # 5.7788 um
print('Ch3 read')
ch4_data  = np.array(data20['SPLASH_CH4_FLUX']) # 7.9958 um 
print('Ch4 read')

Ch1 read
Ch2 read
Ch3 read
Ch4 read


In [21]:
# returns the IRAC channels only at indeces i 
def SelectIRAC(i):
    return ch1_data[i], ch2_data[i], ch3_data[i], ch4_data[i]

In [27]:
# filter data by selection method
ch1_ir, ch2_ir, ch3_ir, ch4_ir = SelectIRAC(agn_irSel_i_ONLY)
ch1_xr, ch2_xr, ch3_xr, ch4_xr = SelectIRAC(agn_xrSel_i_ONLY)
ch1_ix, ch2_ix, ch3_ix, ch4_ix = SelectIRAC(agn_i_BOTH)

# verify 
print('Number of IR sources:\t', len(ch1_ir))
print('Number of Xray sources:\t', len(ch1_xr))
print('Number of Both sources:\t', len(ch1_ix))

Number of IR sources:	 1640
Number of Xray sources:	 1529
Number of Both sources:	 647


# Plot IRAC Colors